# Laptops data analysis

We're given a data on laptops- comprising of data on model number- manufacturer company- RAM- screen size etc.

The primary idea is to clean the data.
We'll perform the follwoing tasks:-
    1. Cleaning the Column heads of the unnecessary spaces and characters.
    2. Convert data stored in certain numerical columns as string-object to int or float.
    3. Splitting columns to new columns with specific information.
    4. Treating columns with null values.

The data can be found here [laptops](https://drive.google.com/file/d/1iF9ZFMUIVRqgjbB368aLzDneaNk7V3GM/view?usp=sharing)

In [2]:
import pandas as pd

In [3]:
laptops = pd.read_csv('laptops.csv',encoding = 'Latin-1')
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


Starting with cleaning the Column heads of the unnecessary spaces and characters.

In [4]:
def refine(string):
    string = string.strip()
    string = string.replace('Operating System' ,'os')
    string = string.replace(' ' ,'_')
    string = string.replace('(' , '')
    string = string.replace(')' , '')
    string = string.lower()
    return string
    
new_columns = []
for c in laptops.columns:
    clean_c = refine(c)
    new_columns.append(clean_c)
    
laptops.columns = new_columns

We can understand from the df.info analysis that columns like RAM and Screen size are purely numerical categories but are still listed as object- string. So to enable numerical computations on these columns , they must be converted to numerical data type float or integer.

In [5]:
laptops["screen_size"] = laptops["screen_size"].str.replace('"','').astype(float)
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

laptops['ram'] = laptops['ram'].str.replace('GB','')
laptops['ram'] = laptops['ram'].astype(int)
laptops.rename({'ram' : 'ram_gb'} , axis =1 , inplace=True)
dtypes = laptops.dtypes

In [6]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
manufacturer          1303 non-null object
model_name            1303 non-null object
category              1303 non-null object
screen_size_inches    1303 non-null float64
screen                1303 non-null object
cpu                   1303 non-null object
ram_gb                1303 non-null int64
storage               1303 non-null object
gpu                   1303 non-null object
os                    1303 non-null object
os_version            1133 non-null object
weight                1303 non-null object
price_euros           1303 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 132.4+ KB


Cleaning the Weight and Price_euros columns.

In [7]:
laptops['weight'] = laptops['weight'].str.replace('kg' ,'')
laptops['weight'] = laptops['weight'].str.replace('s' ,'')
laptops['weight'] =laptops['weight'].astype(float)
laptops.rename({'weight' : 'weight_kg'} , axis =1 , inplace=True)
weight_describe = laptops['weight_kg'].describe()

laptops['price_euros'] = laptops['price_euros'].str.replace(',', '.')
laptops['price_euros'] = laptops['price_euros'].astype(float)

price_describe = laptops['price_euros'].describe()
print(price_describe)
print(weight_describe)

count    1303.000000
mean     1123.686992
std       699.009043
min       174.000000
25%       599.000000
50%       977.000000
75%      1487.880000
max      6099.000000
Name: price_euros, dtype: float64
count    1303.000000
mean        2.038734
std         0.665475
min         0.690000
25%         1.500000
50%         2.040000
75%         2.300000
max         4.700000
Name: weight_kg, dtype: float64


Now we intend to find out the top cpu manufactures. We'll do this by splitting the GPU to extract only the company name and place it in a new column and use value_counts to make our predictions.

In [8]:
laptops["gpu_manufacturer"] = (laptops["gpu"]
                                    .str.split(n=1,expand=True)
                                    .iloc[:,0]
                               )

laptops['cpu_manufacturer'] = laptops['cpu'].str.split(n =1 , expand=True).iloc[:,0]
print(laptops['cpu_manufacturer'].value_counts())

Intel      1240
AMD          62
Samsung       1
Name: cpu_manufacturer, dtype: int64


Intel all the way!

Now i intend to extract the screen resolution from 'screen' column form a seperate column and speed of the laptop(in Ghz) from the cpu column. 

In [9]:
screen_res = laptops["screen"].str.rsplit(n=1, expand=True)
screen_res.columns = ["A", "B"]
screen_res.loc[screen_res["B"].isnull(), "B"] = screen_res["A"]
laptops["screen_resolution"] = screen_res["B"]


laptops['cpu_speed_ghz'] = laptops['cpu'].str.replace('GHz','')
cpu_speed = laptops['cpu_speed_ghz'].str.rsplit(n=1 ,expand= True)
cpu_speed.columns = ['C','D' ]
laptops["cpu_speed_ghz"] = cpu_speed['D']
laptops['cpu_speed_ghz'] =laptops['cpu_speed_ghz'].astype(float)

                     

In [10]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 17 columns):
manufacturer          1303 non-null object
model_name            1303 non-null object
category              1303 non-null object
screen_size_inches    1303 non-null float64
screen                1303 non-null object
cpu                   1303 non-null object
ram_gb                1303 non-null int64
storage               1303 non-null object
gpu                   1303 non-null object
os                    1303 non-null object
os_version            1133 non-null object
weight_kg             1303 non-null float64
price_euros           1303 non-null float64
gpu_manufacturer      1303 non-null object
cpu_manufacturer      1303 non-null object
screen_resolution     1303 non-null object
cpu_speed_ghz         1303 non-null float64
dtypes: float64(4), int64(1), object(12)
memory usage: 173.1+ KB


Examining the 'os' column

In [11]:
print(laptops["os"].value_counts())

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          13
Mac OS          8
Android         2
Name: os, dtype: int64


Mac OS and macOS correspond to the same operating system but are listed differently. Let's deal this by creating a dictionary of the OS to the right coulumn names.

In [12]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}

laptops['os'] = laptops['os'].map(mapping_dict)

In [13]:
print(laptops['os'])

0         macOS
1         macOS
2         No OS
3         macOS
4         macOS
5       Windows
6         macOS
7         macOS
8       Windows
9       Windows
10        No OS
11        No OS
12        macOS
13      Windows
14        macOS
15        macOS
16      Windows
17        macOS
18        No OS
19      Windows
20      Windows
21      Windows
22        No OS
23      Windows
24      Windows
25      Windows
26        macOS
27      Windows
28      Windows
29      Windows
         ...   
1273    Windows
1274    Windows
1275    Windows
1276    Windows
1277      Linux
1278    Windows
1279    Windows
1280    Windows
1281      Linux
1282    Windows
1283    Windows
1284    Windows
1285    Windows
1286    Windows
1287    Windows
1288    Windows
1289    Windows
1290    Windows
1291      Linux
1292    Windows
1293    Windows
1294    Windows
1295      Linux
1296    Windows
1297    Windows
1298    Windows
1299    Windows
1300    Windows
1301    Windows
1302    Windows
Name: os, Length: 1303, 

Now we'll examine if any rows have null values.

In [14]:
print(laptops.isnull().sum())

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight_kg               0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
screen_resolution       0
cpu_speed_ghz           0
dtype: int64


We can infer that the most null values occur in the column os_version.

Let's explore this column in depth.

In [15]:
print(laptops["os_version"].value_counts(dropna=False))

10      1072
NaN      170
7         45
10 S       8
X          8
Name: os_version, dtype: int64


We can also explore values of the other columns in the rows with null values. In this case, the os_version column is closely related to the os column, so we'll look at those values:

In [16]:
os_with_null_v = laptops.loc[laptops["os_version"].isnull(),"os"]
print(os_with_null_v.value_counts())

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64


We can make the following infrences :-
    -Most of the missing values are actually when the laptop doesn't include any OS. This is an important     distinction, because it's not so much that we don't know what the value is, as that there can't be a value.
    -13 of the laptops that come with macOS do not specify the version. Leaning on our knowledge of MacOS, we might know that the full name of macOS used to be Mac OS X, and so we might to fill these values to be more consistent.


In [17]:
mac_os_versions = laptops.loc[laptops["os"] == "macOS", "os_version"]
print(mac_os_versions.value_counts(dropna=False))

NaN    13
X       8
Name: os_version, dtype: int64


Clearly the version for OS macOS is X. So NaN can be replaced with X.

In [18]:
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"

Also the rows with No OS in OS column can be replaced with Version Unknown instead of null.

In [20]:
laptops.loc[ laptops['os'] == 'No OS' ,'os_version'] = 'Version Unknown'
value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
print(value_counts_after)

Linux        62
Chrome OS    27
Android       2
Name: os, dtype: int64


In [22]:
print(laptops["os_version"].value_counts(dropna=False))

10                 1072
NaN                  91
Version Unknown      66
7                    45
X                    21
10 S                  8
Name: os_version, dtype: int64
